## Setup

In [1]:
import MEArec as mr
import numpy as np
import scipy.optimize
import os
import sys
import re
import ast
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import time
from collections import defaultdict
from joblib import Parallel, delayed

import spikeinterface as si
import spikeinterface.core as sc
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface.preprocessing as spre
import spikeinterface.postprocessing as spost
import spikeinterface.widgets as sw
import spikeinterface.comparison as scomp
import spikeinterface.curation as scu

sys.path.append('src')
from src.util_eval import *
from src.util_loc import *

In [ ]:
# Overall parameters
mearec_seed = 42
dead_indices_seeds = [42, 43, 44]
run_id = f'toy_{mearec_seed}'
sorter_name = 'mountainsort4'
days = ['D0', 'D1', 'D2', 'D3', 'D4', 'D5']
methods = ['center_of_mass', 'monopolar_triangulation', 'grid_convolution']
stable_days = 1
dead_electrodes_per_day = 90
gt_toggle = 0 # if 0, use ground truth data as experimental data
correct_radius = 30 # microns

# MEArec parameters
mearec_probe = 'Neuropixels-384'
num_channels = 384
recgen_duration = 60
mearec_noise_level = 10
output_mearec = f'output/{run_id}'

In [ ]:
# Create the template and recording generator for multiple days
cell_folder = mr.get_default_cell_models_folder()
temp_params = mr.get_default_templates_params()
temp_params['n'] = 25
temp_params['probe'] = mearec_probe

for dead_indices_seed_i, dead_indices_seed in enumerate(dead_indices_seeds):    
    output_mearec_seed = f'{output_mearec}/seed_{dead_indices_seed}'
    
    if os.path.exists(f'{output_mearec_seed}/templates.h5'):
        tempgen = mr.load_templates(f'{output_mearec_seed}/templates.h5')
        recgen = mr.load_recordings(f'{output_mearec_seed}/D0/recording.h5')
    
    else:
        temp_params['seed'] = mearec_seed
        tempgen = mr.gen_templates(cell_models_folder=cell_folder, params=temp_params, templates_tmp_folder=None, intraonly=False, 
                                   parallel=True, recompile=False, n_jobs=None, delete_tmp=True, verbose=False)
        mr.save_template_generator(tempgen, f'{output_mearec_seed}/templates.h5')
            
        for day_i, day in enumerate(days):
            rec_params = mr.get_default_recordings_params()
            rec_params['spiketrains']['n_exc'] = 35 # Number of excitatory neurons
            rec_params['spiketrains']['n_inh'] = 15 # Number of inhibitory neurons

            rec_params['spiketrains']['duration'] = recgen_duration # Duration of recording in seconds
            rec_params['recordings']['fs'] = 30000 # Sampling frequency
            rec_params['recordings']['noise_level'] = mearec_noise_level
            rec_params['seeds']['templates'] = mearec_seed

            recgen = mr.gen_recordings(params=rec_params, tempgen=tempgen)
            mr.save_recording_generator(recgen, f'{output_mearec_seed}/{day}/recording.h5')

Setting n_jobs to 20 CPUs
Starting simulation 1/13 - cell: L5_BP_bAC217_1

Starting simulation 2/13 - cell: L5_BTC_bAC217_1

Starting simulation 3/13 - cell: L5_ChC_cACint209_1

Starting simulation 5/13 - cell: L5_LBC_bAC217_1

Starting simulation 9/13 - cell: L5_SBC_bNAC219_1

Starting simulation 8/13 - cell: L5_NGC_bNAC219_1

Starting simulation 10/13 - cell: L5_STPC_cADpyr232_1

Starting simulation 4/13 - cell: L5_DBC_bAC217_1

Starting simulation 13/13 - cell: L5_UTPC_cADpyr232_1

Starting simulation 7/13 - cell: L5_NBC_bAC217_1

Starting simulation 6/13 - cell: L5_MC_bAC217_1

Starting simulation 12/13 - cell: L5_TTPC2_cADpyr232_1

Starting simulation 11/13 - cell: L5_TTPC1_cADpyr232_1

Intracellular simulation: /home/hao-zhao/.config/mearec/1.9.1/cell_models/bbp/L5_BP_bAC217_1
Extracellular simulation: /home/hao-zhao/.config/mearec/1.9.1/cell_models/bbp/L5_BP_bAC217_1
Intracellular simulation: /home/hao-zhao/.config/mearec/1.9.1/cell_models/bbp/L5_BTC_bAC217_1
Extracellular simul

In [ ]:
# Function to process each seed
def process_seed(dead_indices_seed_i, dead_indices_seed):
    output_mearec_seed = f'{output_mearec}/seed_{dead_indices_seed}'
    
    # Dead indices
    if os.path.exists(f'{output_mearec_seed}/dead_indices.pkl'):
        print('Loading dead indices')
        with open(f'{output_mearec_seed}/dead_indices.pkl', 'rb') as f:
            dead_indices = pickle.load(f)
    else:
        print('Creating dead indices')
        np.random.seed(dead_indices_seed)
        
        dead_indices = []
        dead_indices_temp = np.random.choice(range(num_channels), size=dead_electrodes_per_day*(len(days)-stable_days), replace=False)
        for day_i, day in enumerate(days):
            if day_i < stable_days:
                dead_indices.append([])
            else:
                dead_indices.append(dead_indices_temp[:dead_electrodes_per_day*(day_i-1)])
        
        with open(f'{output_mearec_seed}/dead_indices.pkl', "wb") as file:
            pickle.dump(dead_indices, file)

    # Recording data
    recording_mearec = se.MEArecRecordingExtractor(f'{output_mearec_seed}/D0/recording.h5')
    probe = recording_mearec.get_probe()

    recordings = []
    for day_i, day in enumerate(days):
        
        if os.path.exists(f'{output_mearec_seed}/{day}/recording'):
            recordings.append(sc.load_extractor(f'{output_mearec_seed}/{day}/recording').set_probe(probe, in_place=True))
        else:
            recording_mearec = se.MEArecRecordingExtractor(f'{output_mearec_seed}/{day}/recording.h5')
            probe = recording_mearec.get_probe()
            get_recording_noise(recording_mearec, dead_indices[day_i], f'{output_mearec_seed}/{day}/recording')
            recordings.append(sc.load_extractor(f'{output_mearec_seed}/{day}/recording').set_probe(probe, in_place=True))
            

    recording = sc.concatenate_recordings(recordings).set_probe(probe, in_place=True)    
    
    # Sorting data
    sortings_gt = []
    for day_i, day in enumerate(days):
        sortings_gt.append(se.MEArecSortingExtractor(f'{output_mearec_seed}/{day}/recording.h5'))
    sorting_gt = sc.append_sortings(sortings_gt)        
    
    # Waveform extractor data
    wes_gt = []
    for day_i, day in enumerate(days):
        wes_gt.append(si.extract_waveforms(recordings[day_i], sortings_gt[day_i], folder=f'{output_mearec_seed}/{day}/waveforms_gt', ms_before=1, ms_after=2, load_if_exists=True))
        wes_gt[-1].run_extract_waveforms()

    # Calculate template locations
    if os.path.exists(f'{output_mearec_seed}/loc_est_units'):
        print('Loading template localization estimates')
        with open(f'{output_mearec_seed}/loc_est_units/loc_est_units.pkl', 'rb') as f:
            loc_est_units = pickle.load(f)
        with open(f'{output_mearec_seed}/loc_est_units/time_units.pkl', 'rb') as f:
            time_units = pickle.load(f)
            
    else:
        print('Calculating template localization estimates')            
        loc_est_units = {} # First entry is method. Then the entry is a list of locations across days.
        time_units = {}

        for method in methods:
            start_time = time.time()
            loc_est_units[method] = get_unit_loc_est(method, wes_gt)
            time_units[method] = time.time() - start_time
        
        os.makedirs(f'{output_mearec_seed}/loc_est_units')
        pickle.dump(loc_est_units, open(f'{output_mearec_seed}/loc_est_units/loc_est_units.pkl', 'wb'))
        pickle.dump(time_units, open(f'{output_mearec_seed}/loc_est_units/time_units.pkl', 'wb'))        

    # Calculate spike location estimates
    if os.path.exists(f'{output_mearec_seed}/loc_est_spikes'):
        print('Loading spike localization estimates')
        with open(f'{output_mearec_seed}/loc_est_spikes/loc_est_spikes.pkl', 'rb') as f:
            loc_est_spikes = pickle.load(f)
        with open(f'{output_mearec_seed}/loc_est_spikes/time_spikes.pkl', 'rb') as f:
            time_spikes = pickle.load(f)

    else:
        print('Calculating spike localization estimates')
        loc_est_spikes = {} # First entry is method, second is day.
        time_spikes = {}

        for method in methods:
            
            loc_est_spikes[method] = {}
            time_spikes[method] = {}
            for day_i, day in enumerate(days):
                
                start_time = time.time()
                if method == 'monopolar_triangulation':
                    unit_loc_est = spost.compute_spike_locations(wes_gt[day_i], method=method, outputs='by_unit', method_kwargs={'optimizer': 'least_square'})
                else:
                    unit_loc_est = spost.compute_spike_locations(wes_gt[day_i], method=method, outputs='by_unit')
                time_spikes[method][day] = time.time() - start_time
                loc_est_spikes[method][day] = unit_loc_est
        
        os.makedirs(f'{output_mearec_seed}/loc_est_spikes')
        pickle.dump(loc_est_spikes, open(f'{output_mearec_seed}/loc_est_spikes/loc_est_spikes.pkl', 'wb'))
        pickle.dump(time_spikes, open(f'{output_mearec_seed}/loc_est_spikes/time_spikes.pkl', 'wb'))
    
# Use joblib to parallelize the processing of seeds
Parallel(n_jobs=1)(delayed(process_seed)(dead_indices_seed_i, dead_indices_seed) for dead_indices_seed_i, dead_indices_seed in enumerate(dead_indices_seeds))

Creating dead indices
write_binary_recording with n_jobs = 1 and chunk_size = 30000


/home/hao-zhao/anaconda3/envs/haozhao-si-0.100/lib/python3.8/site-packages/numpy/lib/arraysetops.py:587: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar2 = np.asarray(ar2).ravel()
/home/hao-zhao/anaconda3/envs/haozhao-si-0.100/lib/python3.8/site-packages/numpy/lib/arraysetops.py:608: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


write_binary_recording:   0%|          | 0/60 [00:00<?, ?it/s]

write_binary_recording with n_jobs = 1 and chunk_size = 30000


write_binary_recording:   0%|          | 0/60 [00:00<?, ?it/s]

...

extract waveforms shared_memory mono buffer:   0%|          | 0/60 [00:00<?, ?it/s]

localize peaks using grid_convolution:   0%|          | 0/60 [00:00<?, ?it/s]

extract waveforms shared_memory mono buffer:   0%|          | 0/60 [00:00<?, ?it/s]

localize peaks using grid_convolution:   0%|          | 0/60 [00:00<?, ?it/s]

[None, None, None, None, None]